<a href="https://colab.research.google.com/github/mkelly674/Crypto_Clustering_Analysis/blob/main/crypto_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clustering Crypto

In [191]:
!pip install hvplot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [192]:
# Initial imports
import pandas as pd
import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [193]:
# Load the crypto_data.csv dataset.
# YOUR CODE HERE
crypto_df = pd.read_csv("crypto_data.csv",index_col=[0])
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [194]:
# Keep all the cryptocurrencies that are being traded.
# YOUR CODE HERE
crypto_df = crypto_df[crypto_df["IsTrading"]== True]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [195]:
# Remove the "IsTrading" column. 
# YOUR CODE HERE
crypto_df = crypto_df.drop(["IsTrading"], axis = 1)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [196]:
# Remove rows that have at least 1 null value.
# YOUR CODE HERE
crypto_df = crypto_df.dropna()
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [197]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df["TotalCoinsMined"]>0]
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [198]:
# Create a new DataFrame that holds only the cryptocurrencies names.
MinedCoins = crypto_df[['CoinName']]
MinedCoins.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [199]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
# YOUR CODE HERE
crypto_df= crypto_df.drop(["CoinName"],axis = 1)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [200]:
# Use get_dummies() to create variables for text features.
# YOUR CODE HERE
X = pd.get_dummies(crypto_df, columns= ['Algorithm','ProofType'])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [201]:
# Standardize the data with StandardScaler().
# YOUR CODE HERE
X_scaled = StandardScaler().fit_transform(X)
X_scaled

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [202]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)
X_pca

array([[-0.35819299,  0.82113564, -0.14352708],
       [-0.34151613,  0.82132933, -0.1436948 ],
       [ 2.30060236,  1.66485628, -0.27856256],
       ...,
       [ 0.34510311, -2.16324222,  0.06224028],
       [-0.11634983, -1.99394456,  0.11132226],
       [-0.30017129,  0.73458125, -0.06069842]])

In [203]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(data=X_pca, columns = ["PC 1", "PC 2","PC 3"], index = crypto_df.index)
pcs_df

,PC 1,PC 2,PC 3
42,-0.358193,0.821136,-0.143527
404,-0.341516,0.821329,-0.143695
1337,2.300602,1.664856,-0.278563
BTC,-0.135897,-1.188883,0.098000
ETH,-0.146019,-1.923546,0.133130
...,...,...,...
ZEPH,2.459025,0.799877,-0.033961
GAP,-0.356236,0.821032,-0.143546
BDX,0.345103,-2.163242,0.062240
ZEN,-0.116350,-1.993945,0.111322


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [204]:
#getting k and inertia
inertia = []
k = range(1,11)
for i in k:
  kmeanmodel = KMeans(n_clusters= i, random_state = 0).fit(pcs_df)
  inertia.append(kmeanmodel.inertia_)
# Create an elbow curve to find the best value for K.
elbow_data= {"k":k, "inertia":inertia}
df_elbow = pd.DataFrame(elbow_data)

df_elbow.hvplot.line(x="k", y="inertia", title ="Elbow Curve", xticks=k)

/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change fro

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [205]:
# Initialize the K-Means model.
model = KMeans(n_clusters= 4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
pred = model.predict(pcs_df)
pred

/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



array([3, 3, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 3, 3, 0, 3, 0, 0, 3, 3, 0, 0,
       0, 0, 0, 3, 0, 0, 0, 3, 0, 3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 3,
       0, 0, 0, 0, 0, 3, 3, 0, 3, 0, 0, 0, 0, 3, 0, 0, 3, 0, 3, 3, 3, 0,
       0, 0, 3, 3, 3, 3, 3, 0, 0, 0, 3, 3, 0, 3, 0, 3, 3, 0, 0, 0, 0, 3,
       3, 0, 0, 0, 0, 3, 3, 0, 3, 3, 0, 0, 0, 3, 0, 3, 3, 0, 3, 0, 3, 0,
       3, 0, 3, 3, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 3,
       0, 3, 0, 0, 3, 0, 3, 0, 3, 3, 0, 0, 3, 0, 0, 3, 3, 0, 3, 0, 3, 3,
       3, 0, 0, 0, 0, 3, 3, 3, 0, 3, 0, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3,
       3, 0, 3, 0, 3, 3, 0, 3, 0, 3, 3, 0, 3, 0, 3, 0, 0, 0, 3, 3, 3, 3,
       0, 3, 3, 3, 3, 3, 0, 0, 3, 3, 0, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3,
       3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 0, 0, 3, 3, 3, 3, 0, 3, 0, 3,
       0, 0, 3, 0, 0, 3, 0, 0, 3, 0, 3, 3, 3, 0, 3, 3, 0, 3, 3, 3, 3, 3,
       3, 3, 0, 3, 0, 3, 3, 3, 3, 0, 3, 0, 3, 0, 0, 0, 0, 3, 0, 3, 3, 0,
       3, 0, 0, 0, 3, 0, 3, 0, 0, 0, 3, 0, 3, 0, 3,

In [206]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df,pcs_df], axis =1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName'] = MinedCoins['CoinName']

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['Class'] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.358193,0.821136,-0.143527,42 Coin,3
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.341516,0.821329,-0.143695,404Coin,3
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.300602,1.664856,-0.278563,EliteCoin,3
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.135897,-1.188883,0.098000,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.146019,-1.923546,0.133130,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.177764,-1.219435,0.040911,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.383164,1.246099,-0.177966,Dash,3
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.132139,-2.071328,0.097576,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.144459,-1.923635,0.133114,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.116349,-1.993945,0.111322,ZCash,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [207]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x='PC 1',
    y='PC 2',
    z='PC 3',
    hover_name = 'CoinName',
    hover_data = ['Algorithm'],
    color='Class',
    symbol='Class',
    width=800,)
fig.update_layout(legend=dict(x=0,y=1))
fig.show()

In [208]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class'], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [209]:
# Print the total number of tradable cryptocurrencies.
print(len(clustered_df['CoinName']))

532


In [217]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
X_scaler=MinMaxScaler().fit_transform(clustered_df[['TotalCoinSupply','TotalCoinsMined']])
X_scaler

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [218]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
X_scaler = clustered_df.loc[:,('TotalCoinSupply', 'TotalCoinsMined')]

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
X_scaler["CoinName"] = clustered_df["CoinName"]

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
X_scaler["Class"] = clustered_df["Class"]

X_scaler.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,42,4.199995e+01,42 Coin,3
404,532000000,1.055185e+09,404Coin,3
1337,314159265359,2.927942e+10,EliteCoin,3
BTC,21000000,1.792718e+07,Bitcoin,0
ETH,0,1.076842e+08,Ethereum,0
LTC,84000000,6.303924e+07,Litecoin,0
DASH,22000000,9.031294e+06,Dash,3
XMR,0,1.720114e+07,Monero,0
ETC,210000000,1.133597e+08,Ethereum Classic,0
ZEC,21000000,7.383056e+06,ZCash,0


In [222]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
X_scaler.hvplot.scatter(
    x="TotalCoinsMined", 
    y="TotalCoinSupply", 
    by="Class", 
    hover_cols="CoinName",
    line_color='black',
    size=75
   )

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)